In [91]:
#Import Library Google API & Tools
from googleapiclient.discovery import build
from dateutil import parser
import pandas as pd
from IPython.display import JSON
import json

#Data Visualisasion 
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

In [92]:
jsonFile = open('./credential.json','r');
data = json.load(jsonFile);
tokenAPI = data['youtubeToken'];
channelId = data['channelId'];
jsonFile.close()

In [93]:
#Variable Credentials
apiYoutube = tokenAPI
idYoutube = channelId
serviceName = 'youtube'
apiVersion = 'v3'

In [94]:
#Create API Youtube Channel
youtube = build(serviceName,apiVersion,developerKey=apiYoutube);

In [95]:
#Function To Get Data API Youtube
def get_channel_stats(youtube, channel_ids):
    """
    Get channel statistics: title, subscriber count, view count, video count, upload playlist
    Params:
    
    youtube: the build object from googleapiclient.discovery
    channels_ids: list of channel IDs
    
    Returns:
    Dataframe containing the channel statistics for all channels in the provided list: title, subscriber count, view count, video count, upload playlist
    
    """
    all_data = []
    request = youtube.channels().list(
                part='snippet,contentDetails,statistics',
                id=','.join(channel_ids))
    response = request.execute() 
    
    for i in range(len(response['items'])):
        data = dict(channelName = response['items'][i]['snippet']['title'],
                    subscribers = response['items'][i]['statistics']['subscriberCount'],
                    views = response['items'][i]['statistics']['viewCount'],
                    totalVideos = response['items'][i]['statistics']['videoCount'],
                    playlistId = response['items'][i]['contentDetails']['relatedPlaylists']['uploads'])
        all_data.append(data)
    
    return pd.DataFrame(all_data)

def get_video_ids(youtube, playlist_id):
    """
    Get list of video IDs of all videos in the given playlist
    Params:
    
    youtube: the build object from googleapiclient.discovery
    playlist_id: playlist ID of the channel
    
    Returns:
    List of video IDs of all videos in the playlist
    
    """
    
    request = youtube.playlistItems().list(
                part='contentDetails',
                playlistId = playlist_id,
                maxResults = 50)
    response = request.execute()
    
    video_ids = []
    
    for i in range(len(response['items'])):
        video_ids.append(response['items'][i]['contentDetails']['videoId'])
        
    next_page_token = response.get('nextPageToken')
    more_pages = True
    
    while more_pages:
        if next_page_token is None:
            more_pages = False
        else:
            request = youtube.playlistItems().list(
                        part='contentDetails',
                        playlistId = playlist_id,
                        maxResults = 50,
                        pageToken = next_page_token)
            response = request.execute()
    
            for i in range(len(response['items'])):
                video_ids.append(response['items'][i]['contentDetails']['videoId'])
            
            next_page_token = response.get('nextPageToken')
        
    return video_ids

def get_video_details(youtube, video_ids):
    """
    Get video statistics of all videos with given IDs
    Params:
    
    youtube: the build object from googleapiclient.discovery
    video_ids: list of video IDs
    
    Returns:
    Dataframe with statistics of videos, i.e.:
        'channelTitle', 'title', 'description', 'tags', 'publishedAt'
        'viewCount', 'likeCount', 'favoriteCount', 'commentCount'
        'duration', 'definition', 'caption'
    """
        
    all_video_info = []
    
    for i in range(0, len(video_ids), 50):
        request = youtube.videos().list(
            part="snippet,contentDetails,statistics",
            id=','.join(video_ids[i:i+50])
        )
        response = request.execute() 

        for video in response['items']:
            stats_to_keep = {'snippet': ['channelTitle', 'title', 'description', 'tags', 'publishedAt'],
                             'statistics': ['viewCount', 'likeCount', 'favouriteCount', 'commentCount'],
                             'contentDetails': ['duration', 'definition', 'caption']
                            }
            video_info = {}
            video_info['video_id'] = video['id']

            for k in stats_to_keep.keys():
                for v in stats_to_keep[k]:
                    try:
                        video_info[v] = video[k][v]
                    except:
                        video_info[v] = None

            all_video_info.append(video_info)
            
    return pd.DataFrame(all_video_info)

def get_comments_in_videos(youtube, video_ids):
    """
    Get top level comments as text from all videos with given IDs (only the first 10 comments due to quote limit of Youtube API)
    Params:
    
    youtube: the build object from googleapiclient.discovery
    video_ids: list of video IDs
    
    Returns:
    Dataframe with video IDs and associated top level comment in text.
    
    """
    all_comments = []
    
    for video_id in video_ids:
        try:   
            request = youtube.commentThreads().list(
                part="snippet,replies",
                videoId=video_id
            )
            response = request.execute()
        
            comments_in_video = [comment['snippet']['topLevelComment']['snippet']['textOriginal'] for comment in response['items'][0:10]]
            comments_in_video_info = {'video_id': video_id, 'comments': comments_in_video}

            all_comments.append(comments_in_video_info)
            
        except: 
            # When error occurs - most likely because comments are disabled on a video
            print('Could not get comments for video ' + video_id)
        
    return pd.DataFrame(all_comments) 

In [96]:
#Showing Stats Data Channel Youtube
channelStats = get_channel_stats(youtube,idYoutube);
channelStats

,channelName,subscribers,views,totalVideos,playlistId
0,Caveine,62700,11278330,145,UUpw2MGdcV_a-QyDzXTLitDg
1,Dea Afrizal,222000,14522144,352,UUU7YluxOYon-yofPxfGHVog


In [97]:
#Playlist ID
playlistId = 'PLr1UGQutinsax9NDyzVb3DmnvGvkAtaA4'
videoId = get_video_ids(youtube,playlistId);
videoId

['RMe57c4ZhyQ',
 'Xn-qDR6eWYg',
 'rQ3REj6bkhc',
 '0d3hQjTIeCA',
 '4QnP-bCKDxI',
 'kWT9q6t_1vc',
 '2IFCgh9PwNk',
 '597jV43e9FQ',
 'ReCIlI3bu9k',
 '7n02M1k5-oM',
 'kQPnC77Q8AY',
 'q9DwMnKbvEY',
 'latFYMUP_YI',
 'c_iKt77xeKc',
 '1kKngGSyNMc',
 '617zx0tseyU',
 'RbUlQWyMgXc',
 'BgItJZIebyc',
 'hJvdEyLdAe0',
 'wP7ameZmZgI',
 'ADrOCogg-W4',
 'fdD2_Hm3Szc',
 '5-3HwZh_ZtA',
 'hUJD2_rFUS8',
 'u-VQF-7XFIc',
 'WWHEsIAzQn4',
 'Fut-LIIUna0',
 'K1gPUb8xgUM',
 'I6HcF_CmC8E',
 'jYQWa-Ma-Ug',
 'Mc3CiGxiOOU',
 'kDa1LT_f_D8',
 '8WcqhLIJmcs',
 'Cch-S_YB574',
 'BDhGbwrvq7k',
 'IICOh4HVhV4',
 'EEqSF319v8g',
 'hVLOMweWm6M',
 'n7hRwysA218',
 '7QSakGsj1To',
 'W0qfN5U6wd0',
 'nhYrxZnYeuk',
 '8Z_e0gSNlDs',
 '9gMilO-_hjE',
 '3CGGWRGqlp4',
 'RzbYzGrdyoA',
 'OIHRWLYgQbA',
 'p_PJrGR2QNM',
 'Xp6SG5uN0kA',
 'wIKcvFSaOAQ',
 'n3j7qFSS7oc',
 'cTN_IhZo5u4',
 '9zN1Ump5NVU',
 'vgTbqSbgTpc',
 'QXR4KsvBx7w',
 'OO5W5Ytpe-s',
 'B29cBRa7FcA',
 '-D7it2EqlEI',
 'Flg3FKWZsv8',
 'pfcQy1O-xlc',
 'RGA7muLYT_M',
 'AZAxYnxS4zU',
 'ASn2xO

In [98]:
#Get Detail Video from Video ID
videoYoutubeDetail = get_video_details(youtube,videoId);
videoYoutubeDetail.to_csv(r'data.csv')
videoYoutubeDetail

,video_id,channelTitle,title,description,tags,publishedAt,viewCount,likeCount,favouriteCount,commentCount,duration,definition,caption
0,RMe57c4ZhyQ,Caveine,parkir pinggir jalan nyusahin orang,semua video yang gw bikin buat becanda doang y...,"[tiktok meresahkan, keresahan warga tiktok, ke...",2023-04-11T09:00:23Z,5151,553,None,79,PT57S,hd,false
1,Xn-qDR6eWYg,Caveine,yang ngerokok dijalan G4Y!,semua video yang gw bikin buat becanda doang y...,"[tiktok meresahkan, keresahan warga tiktok, ke...",2023-04-08T09:00:19Z,6301,628,None,82,PT1M,hd,false
2,rQ3REj6bkhc,Caveine,"antri gaes ada motovloger, gaboleh nyalip!",#motovlog #motovlogger\nsemua video yang gw bi...,"[tiktok meresahkan, keresahan warga tiktok, ke...",2023-04-03T09:00:34Z,9471,883,None,107,PT59S,hd,false
3,0d3hQjTIeCA,Caveine,Waktu Teraweh Malah Push Rank,semua video yang gw bikin buat becanda doang y...,"[tiktok meresahkan, keresahan warga tiktok, ke...",2023-04-02T09:00:02Z,7543,729,None,68,PT1M,hd,false
4,4QnP-bCKDxI,Caveine,Acara Masak Tapi Bawa Bawa Ras,semua video yang gw bikin buat becanda doang y...,"[tiktok meresahkan, keresahan warga tiktok, ke...",2023-03-29T09:00:03Z,8531,722,None,115,PT1M,hd,false
...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,aGve70-OPJI,Caveine,OSIS HANYA ALAT😱 AFFAH IYA?,semua video yang gw bikin buat becanda doang y...,"[tiktok meresahkan, keresahan warga tiktok, ke...",2022-08-22T11:59:56Z,30405,1659,None,116,PT57S,hd,false
66,FuAt3p5_ReE,Caveine,BOCIL yang SI PALING BEGADANG,semua video yang gw bikin buat becanda doang y...,"[meme indonesia, bocil kematian, bocil begadan...",2022-08-16T08:08:43Z,22233,1190,None,159,PT57S,hd,false
67,roIzN5e4QR8,Caveine,ISTRIMU KARTUN?? Yok Berobat 🚑,semua video yang gw bikin buat becanda doang y...,"[meme indonesia, tiktok meresahkan, keresahan ...",2022-08-19T14:28:53Z,25275,1328,None,113,PT1M9S,hd,false
68,uD-DAuumCaM,Caveine,Tips menghadapi player TOXIC,semua video yang gw bikin buat becanda doang y...,"[mobile legends, laporkan akun toxic!!! banned...",2022-08-05T07:32:25Z,16890,1098,None,49,PT2M23S,hd,false


In [99]:
#Get Comment From Videos
videoComments = get_comments_in_videos(youtube,videoId)
videoComments

Could not get comments for video 8Z_e0gSNlDs
Could not get comments for video AZAxYnxS4zU


,video_id,comments
0,RMe57c4ZhyQ,"[mobil elit. parkiran sulit, ., mgkn batu buat..."
1,Xn-qDR6eWYg,"[Aku datang kembali 🙂, Keren kali,menurut mere..."
2,rQ3REj6bkhc,"[Bs bayangin kl emak emak nyalip motovlogger, ..."
3,0d3hQjTIeCA,[Setan berkata:enak banget ga dibelenggu 🤬🤬🤬🤬🤬...
4,4QnP-bCKDxI,[INI KONTES MASAK BUKAN KONTES PENAMPILAN KALO...
...,...,...
65,aGve70-OPJI,"[Gweh banget bangh itu ayanokoji, Mata lo gw c..."
66,FuAt3p5_ReE,"[Tapi gua tidur jam 3, Begadang cuma karna Gab..."
67,roIzN5e4QR8,"[Gw pas minum obat kok istri gw ilang?, Mantap..."
68,uD-DAuumCaM,"[Kukira langkah ketiga itu uninstall gamenya, ..."
